In [5]:
"""
experiments.ipynb

File for running lottery ticket experiments.

Authors: Jordan Bourdeau, Casey Forey
Date Created: 3/8/24
"""
%load_ext tensorboard
import copy
import functools
from importlib import reload
import tensorflow as tf

from src.harness import constants as C
from src.harness import experiment
from src.harness import history
from src.harness import paths
from src.harness import pruning
from src.harness import rewind
from src.harness import training as train
from tensorflow.python.client import device_lib 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [8]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(device_lib.list_local_devices())

Num GPUs Available:  0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2527946631643703092
xla_global_id: -1
]


## Run Experiments

In [9]:
reload(C)
reload(experiment)
reload(paths)
reload(pruning)
reload(rewind)
reload(train)

experiment_directory: str = 'testing_experiment'
experiment_summary: history.ExperimentSummary = experiment.run_experiments(
    5, 
    experiment_directory,
    functools.partial(experiment.get_mnist_lenet_300_100_experiment_parameters, 0.2, 0.65),
    experiment.run_iterative_pruning_experiment,
)

Directory 'testing_experiment' already exists.
Directory 'models\model_0\initial\weights' already exists.
Directory 'models\model_0\initial\masks' already exists.
Step 0 of Iterative Magnitude Pruning
Epoch 1 Train Loss: 0.231, Train Accuracy: 0.929, Test Loss: 0.145, Test Accuracy: 0.954
Epoch 2 Train Loss: 0.105, Train Accuracy: 0.968, Test Loss: 0.142, Test Accuracy: 0.958
Epoch 3 Train Loss: 0.074, Train Accuracy: 0.976, Test Loss: 0.133, Test Accuracy: 0.960
Epoch 4 Train Loss: 0.063, Train Accuracy: 0.981, Test Loss: 0.114, Test Accuracy: 0.969
Epoch 5 Train Loss: 0.057, Train Accuracy: 0.982, Test Loss: 0.121, Test Accuracy: 0.968
Epoch 6 Train Loss: 0.050, Train Accuracy: 0.984, Test Loss: 0.113, Test Accuracy: 0.973
